In [1]:
import pandas as pd
import numpy as np
import requests
import urllib
import re
import string
from bs4 import BeautifulSoup

In [2]:
#Open the page with all albums.
main_page = 'https://www.azlyrics.com/a/atmosphere.html'
try:
    content = urllib.request.urlopen(main_page).read()
    main_soup = BeautifulSoup(content, 'html.parser')
except Exception as e:
    print('Error'+str(e))
#Pull the list of albums on the page and the associated IDs.
album_list = main_soup.find(id='listAlbum')
albums = [name.string.strip('\"') for name in album_list.findAll('b')]
album_ids = [number.get('id') for number in album_list.findAll(['a']) if len(number)==0]

In [3]:
#Create a dataframe containing each album and the ID.
albumdf = pd.DataFrame(data={'AlbumName': albums, 'AlbumID': album_ids})
albumdf

,AlbumID,AlbumName
0,7391,Overcast!
1,5001,Lucy Ford: The Atmosphere EPs
2,5005,God Loves Ugly
3,3262,Seven's Travels
4,7592,Headshots: SE7EN
5,5146,You Can't Imagine How Much Fun We're Having
6,7187,Happy Clown Bad Dub 8 / Fun EP
7,6298,Sad Clown Bad Summer #9
8,7048,Sad Clown Bad Fall #10
9,7050,Sad Clown Bad Winter #11


In [4]:
#Need to implement loop/vectorize (if possible) so it can be done on every album.
album_dicts = dict.fromkeys(albumdf['AlbumID'])
for i,j in enumerate(albumdf['AlbumID']):
    #Find the album.
    findalbum = album_list.findAll(class_='album')[i]
    #Initialize song list.
    songs = []
    #Find where the song titles begin.
    title = findalbum.findNext('a', text=True)
    #Loop through the list until reaching the next album, where there is an ID field.
    if i < len(albumdf)-1:
        while title.get('id') != albumdf['AlbumID'][i+1]:
            songs.append(title.text)
            title = title.findNext('a')
        album_dicts[j] = songs
    else:
        while title.text != 'A Long Hello':
            songs.append(title.text)
            title = title.findNext('a')
        album_dicts[j] = songs

In [5]:
#Add song counts to the albumdf.
albumdf['Songs'] = albumdf['AlbumID'].map(len)

In [6]:
#Get list of all songs.
all_songs = [name for sublist in album_dicts.values() for name in sublist]
songdf = pd.DataFrame({'SongName': all_songs})
songdf['SongName']

0                           Like A Fire
1                                 Ringo
2                                 Besos
3                             Pure Evil
4                               Perfect
5                         Seismic Waves
6                           Next To You
7      The Shit That We've Been Through
8                When The Lights Go Out
9                             No Biggie
10                           Everything
11                     Chasing New York
12                                Sugar
13                        Fishing Blues
14                      Won't Look Back
15              Anybody That I've Known
16                        Still Be Here
17               Until The Nipples Gone
18                    The Major Leagues
19                                Scalp
20                         The Best Day
21                         Americareful
22                                 Hope
23                       The Loser Wins
24                              Shotgun


In [7]:
#Function to clean up the song names.
def clean_names(string):
    pattern = re.compile('[\W_]+')
    string = pattern.sub('', string.lower())
    return string

In [8]:
#Function to match albums and songs.
album_id_values = albumdf['AlbumID'].tolist()
def matchAlbum(name):
    for k in album_id_values:
        if name in album_dicts[k]:
            return k

In [9]:
#Create a new column with the cleaned up names.
songdf['CleanedName'] = songdf['SongName'].map(clean_names)
#Create column of album IDs.
songdf['AlbumID'] = songdf['SongName'].map(matchAlbum)
#And another for the different song pages.
songdf['SongPage'] = songdf['CleanedName'].apply(
    lambda name: 'https://www.azlyrics.com/lyrics/atmosphere/'+name+'.html')
songdf

,SongName,CleanedName,AlbumID,SongPage
0,Like A Fire,likeafire,43755,https://www.azlyrics.com/lyrics/atmosphere/lik...
1,Ringo,ringo,43755,https://www.azlyrics.com/lyrics/atmosphere/rin...
2,Besos,besos,43755,https://www.azlyrics.com/lyrics/atmosphere/bes...
3,Pure Evil,pureevil,43755,https://www.azlyrics.com/lyrics/atmosphere/pur...
4,Perfect,perfect,43755,https://www.azlyrics.com/lyrics/atmosphere/per...
5,Seismic Waves,seismicwaves,43755,https://www.azlyrics.com/lyrics/atmosphere/sei...
6,Next To You,nexttoyou,43755,https://www.azlyrics.com/lyrics/atmosphere/nex...
7,The Shit That We've Been Through,theshitthatwevebeenthrough,43755,https://www.azlyrics.com/lyrics/atmosphere/the...
8,When The Lights Go Out,whenthelightsgoout,43755,https://www.azlyrics.com/lyrics/atmosphere/whe...
9,No Biggie,nobiggie,43755,https://www.azlyrics.com/lyrics/atmosphere/nob...


In [10]:
#Next, get a lexicon for every song by iterating through the pages

In [10]:
def lyricRetrieval(givenurl):
    songurl = givenurl
    try:
        song_content = urllib.request.urlopen(songurl).read()
        song_soup = BeautifulSoup(song_content, 'html.parser')
    except Exception as e:
        print('Error'+str(e))
    mess = song_soup.findAll('b', text=True)[1].findNext('div').text
    pattern = re.compile('[\W_]+')
    new_mess = pattern.sub(' ', mess.lower().replace("'", ""))
    return new_mess

In [12]:
#Faster way to do this, but crashes the site. Needed to slow down the requests.
#songdf['Lyrics'] = songdf['SongPage'].map(lyricRetrieval)
import time

lyrics = []
for add in songdf['SongPage']:
    print(add)
    lyrics.append(lyricRetrieval(add))
    time.sleep(1+np.random.randn())

https://www.azlyrics.com/lyrics/atmosphere/likeafire.html
Error''


UnboundLocalError: local variable 'song_soup' referenced before assignment

In [ ]:
#The above resulted in IP blocking from multiple locations. Went to Genius with the data from here.

In [28]:
#AZlyrics seems to be blocking any IP address I run this from, even with a delay built in. Use Genius?

In [78]:
#Use this to retrieve the relevant song page where the lyrics are
def songInfoRetrieve(title, artist_name):
    site_url = 'http://api.genius.com'
    headers = {'Authorization': 'Bearer ' + 'IfomKosHgz_77SarGL77k2HXxxSzTQs9Q4ru6GvcgYGCfYwkqhFNdSHwD1icClqh'}
    search_url = site_url + '/search'
    params = {'q': title + ' ' + artist_name}
    response = requests.get(search_url, params=params, headers=headers)

    json = response.json()
    remote_song_info = None

    for hit in json['response']['hits']:
        if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            break
    if remote_song_info:
        song_url = remote_song_info['result']['url']
        return (song_url, response)
    else:
        return ('No URL found', response)

In [123]:
#Use this to retrieve the lyrics and clean up any non-alphanumeric characters
pattern = re.compile('[\W_]+')
def scrapeSongUrl(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    messy_lyrics = html.find('div', class_='lyrics').get_text()
    #pattern = re.compile('[\W_]+')
    lyrics = pattern.sub(' ', messy_lyrics.lower().replace("\'", ""))
    return lyrics

In [75]:
#Ok, have the lyric retrieval working. Need to setup storage, and cleaning.

#Use earlier df to keep the songs and album ids together.
geniusdf = songdf[['SongName', 'AlbumID']]

In [121]:
#Create status code and url columns. Can check and remove any non-existent urls based on status codes.
geniusdf['SongResponse'] = geniusdf['SongName'].apply(lambda title: songInfoRetrieve(title, 'Atmosphere')[1].status_code)
geniusdf['SongURL'] = geniusdf['SongName'].apply(lambda title: songInfoRetrieve(title, 'Atmosphere')[0])

In [138]:
#Check if any songs are missing from the API
geniusdf[geniusdf['SongURL'] == 'No URL found']
#Remove them from the dataset
geniusdf.drop([105, 162], inplace=True)
geniusdf.reset_index(drop=True, inplace=True)

,SongName,AlbumID,SongResponse,SongURL
160,That Night,5146,200,https://genius.com/Atmosphere-that-night-lyrics
161,Little Man,5146,200,https://genius.com/Atmosphere-little-man-lyrics
162,Like The Rest Of Us,6684,200,https://genius.com/Atmosphere-like-the-rest-of...
163,Puppets,6684,200,https://genius.com/Atmosphere-puppets-lyrics
164,The Skinny,6684,200,https://genius.com/Atmosphere-the-skinny-lyrics
165,Dreamer,6684,200,https://genius.com/Atmosphere-dreamer-lyrics
166,Shoulda Known,6684,200,https://genius.com/Atmosphere-shoulda-known-ly...
167,You,6684,200,https://genius.com/Atmosphere-always-coming-ba...
168,Painting,6684,200,https://genius.com/Atmosphere-painting-lyrics
169,Your Glass House,6684,200,https://genius.com/Atmosphere-your-glasshouse-...


In [139]:
#Every song will have a url, so we can now grab the data from each page.
geniusdf['SongLyrics'] = geniusdf['SongURL'].map(scrapeSongUrl)

In [140]:
geniusdf

,SongName,AlbumID,SongResponse,SongURL,SongLyrics
0,Like A Fire,43755,200,https://genius.com/Atmosphere-like-a-fire-lyrics,intro a recording its like hi this is a recor...
1,Ringo,43755,200,https://genius.com/Atmosphere-ringo-lyrics,verse 1 i think im still drunk from last nigh...
2,Besos,43755,200,https://genius.com/Atmosphere-besos-lyrics,verse 1 slug 1up never still if the sun up it...
3,Pure Evil,43755,200,https://genius.com/Atmosphere-pure-evil-lyrics,chorus i b e no no no no no oh no no no i don...
4,Perfect,43755,200,https://genius.com/Atmosphere-perfect-lyrics,hook 1 fall from the sky just to learn how to...
5,Seismic Waves,43755,200,https://genius.com/Atmosphere-seismic-waves-ly...,verse 1 armageddon started i guess it wasnt t...
6,Next To You,43755,200,https://genius.com/Atmosphere-next-to-you-lyrics,hook dem atlas darlin you dont even know im l...
7,The Shit That We've Been Through,43755,200,https://genius.com/Atmosphere-the-shit-that-we...,verse 1 slug seven in the morning locked the ...
8,When The Lights Go Out,43755,200,https://genius.com/Atmosphere-when-the-lights-...,verse 1 slug do you remember that knife you w...
9,No Biggie,43755,200,https://genius.com/Atmosphere-no-biggie-lyrics,verse slug she said no biggie territorial eno...


In [141]:
#Save the data
geniusdf.to_csv('atmosphere_song_lyrics', header=True)
albumdf.to_csv('atmosphere_album_data', header=True)